In [1]:
pip install pandas beautifulsoup4 requests groq openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
from groq import Groq
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

# --- 1. Setup Grok Client ---
client = Groq(api_key=os.getenv("GROK_API_KEY"))  # 🔐 Load API key from environment variable


# --- 2. Extract all URLs from the Google Sheet ---
def extract_urls_from_sheet(sheet_url):
    csv_url = sheet_url.replace("/edit?usp=sharing", "/export?format=csv")
    df = pd.read_csv(csv_url)
    url_column = df.columns[df.apply(lambda col: col.astype(str).str.startswith("http")).any()].tolist()
    return df[url_column[0]].dropna().tolist() if url_column else []

# --- 3. Extract visible text from a web page ---
def extract_text(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(r.text, 'html.parser')
        for tag in soup(['script', 'style', 'header', 'footer', 'nav']):
            tag.decompose()
        return ' '.join(soup.stripped_strings)
    except Exception as e:
        return f"Error extracting from {url}: {e}"

# --- 4. Extract biomarker info using Grok ---
def grok_extract_biomarker(text):
    prompt = f"""
Extract the following biomedical information from the text below:

Text:
\"\"\"{text}\"\"\"

Extract if available:
- Biomarker Name
- Associated Disease(s)
- Application (e.g., diagnosis, prognosis)
- Biological Source (e.g., blood, urine)
- Mechanism / Pathway
- Other Relevant Terms

Return response as JSON.
"""
    try:
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama-3.3-70b-versatile",
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        return json.dumps({"error": str(e)})

# --- 5. Run everything ---
def main():
    SHEET_URL = "https://docs.google.com/spreadsheets/d/1gnFSagLXtFyx2UgNa4oSPa4PV12R-LROPHHFFj2ICts/edit?usp=sharing"
    urls = extract_urls_from_sheet(SHEET_URL)

    all_results = []

    for url in urls:
        print(f"🔍 Scraping: {url}")
        text = extract_text(url)

        if text.startswith("Error"):
            all_results.append({"URL": url, "Error": text})
            continue

        llm_response = grok_extract_biomarker(text)
        try:
            result = json.loads(llm_response)
            result["URL"] = url
        except:
            result = {"URL": url, "LLM_Output": llm_response, "ParseError": True}

        all_results.append(result)
        time.sleep(1)

    df_out = pd.DataFrame(all_results)
    df_out.to_excel("biomarker_data.xlsx", index=False)
    print("✅ Data saved to biomarker_data.xlsx")

if __name__ == "__main__":
    main()


🔍 Scraping: https://my.clevelandclinic.org/health/articles/23922-what-is-cholesterol
🔍 Scraping: https://my.clevelandclinic.org/health/articles/11920-cholesterol-numbers-what-do-they-mean
🔍 Scraping: https://www.webmd.com/cholesterol-management/understanding-numbers
🔍 Scraping: https://www.heart.org/en/health-topics/cholesterol/hdl-good-ldl-bad-cholesterol-and-triglycerides
🔍 Scraping: https://medlineplus.gov/cholesterol.html#:~:text=What%20is%20cholesterol%3F,all%20the%20cholesterol%20it%20needs
✅ Data saved to biomarker_data.xlsx
